## Koalas fails with the NYC Taxi
Again OOM error. No matter how much I limited the number of rows, it kept happening. I also tried multiple columns. Nothing worked, so I used a smaller dataset (generally, I want to use big datasets because these frameworks specialize there).

In [1]:
import pandas as pd
import numpy as np
import collections.abc
# Koalas needs these mappings
collections.Iterable = collections.abc.Iterable
collections.Callable = collections.abc.Callable
import databricks.koalas as ks
import utils

# It seems you need to set this option for performance reasons.
# See: https://github.com/databricks/koalas/issues/1769 (it seems the issue is not only related to apply())
ks.set_option('compute.default_index_type', 'distributed')

koalas_df = ks.read_csv('../datasets/iris.csv')
pandas_df = pd.read_csv('../datasets/iris.csv')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/25 09:20:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Koalas fails

Runs out of memory again with `pickup_longitude`. See `value_counts.ipynb`.
Also, we have to use `.to_numpy()`, which I guess makes sense because numpy doesn't understand Koalas.

In [2]:
%%time_cell
x = np.sum(koalas_df['sepal_width'].to_numpy())
print(x)

458.6


In [3]:
koalas_time = _TIMED_CELL
print(f"Koalas time: {koalas_time:.1f}s")

Koalas time: 0.2s


In [4]:
%%time_cell
y = np.sum(pandas_df['sepal_width'])
print(y)

458.6


In [5]:
pandas_time = _TIMED_CELL
print(f"Pandas time: {pandas_time:.3f}s")

Pandas time: 0.001s


In [6]:
slowdown = koalas_time / pandas_time
utils.print_md(f"### Koalas is {slowdown:.1f}x slower.")

### Koalas is 190.4x slower.

In [7]:
assert x == y